In [1]:
#Use FFTDock to dock ligands into AlphaFold models with FAD cofactor
#Inputs:
    #cofactor/pdb_with_fad/tropb.pdb
    #dock/ligands/2.pdb
    #dock/ligands/2.str #CGenFF stream file
    #toppar
#Outputs:
    #dock/grid/tropb_fftdock_3.bin
    #dock/poses/tropb_2_fftdock
    #dock/scores/tropb_2_fftdock.csv

In [2]:
import os 
import re
import pandas as pd
import numpy as np
os.environ['CHARMM_LIB_DIR'] = "/home/azamh/charmm-dev/lib"

# These are a subset of the pycharmm modules that were installed when
# pycharmm was installed in your python environment
import pycharmm
import pycharmm.generate as gen
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.nbonds as nbonds
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.select as select
import pycharmm.shake as shake
import pycharmm.settings as settings
import pycharmm.grid as grid
import pycharmm.charmm_file as charmm_file
from pycharmm.select_atoms import SelectAtoms
from pycharmm.lingo import charmm_script

from pycharmm.lib import charmm as libcharmm

os.chdir('/home/azamh/demo/seq_struct_func/dock')

[gollum153][[58604,1],0][../../../../../openmpi-3.1.2/opal/mca/btl/openib/btl_openib_component.c:1671:init_one_device] error obtaining device attributes for mlx5_0 errno says Protocol not supported
--------------------------------------------------------------------------

  Local host:   gollum153
  Local device: mlx5_0
--------------------------------------------------------------------------


In [3]:
#Arguments
protein = 'tropb'
ligand = '2'
toppardir = '../toppar'
liganddir = './ligands'
proteindir = '../cofactor/pdb_with_fad'
dockdir = f'poses/{protein}_{ligand}_fftdock'
os.makedirs(dockdir, exist_ok=True)

## Read in grid box information
xcen = 57.135381
ycen = 45.1752381
zcen = 46.0617619
maxlen = 12

In [4]:
## Read in the topology and parameter file 
settings.set_bomb_level(-1)
read.rtf(os.path.join(toppardir, 'top_all36_prot.rtf'))
read.rtf(os.path.join(toppardir,'top_all36_cgenff.rtf'), append = True)
read.rtf(os.path.join(toppardir,'probes.rtf'), append = True)
read.prm(os.path.join(toppardir, 'par_all36m_prot.prm'), flex = True)
read.prm(os.path.join(toppardir, 'par_all36_cgenff.prm'), append = True, flex = True)
read.prm(os.path.join(toppardir, 'probes.prm'), append = True, flex = True)
settings.set_bomb_level(0)
charmm_script(f'stream {os.path.join(liganddir, ligand)}.str')
charmm_script(f'stream {os.path.join(toppardir, "st2_fadh.str")}')

  
 CHARMM>     read rtf card -
 CHARMM>     name ../toppar/top_all36_prot.rtf
 VOPEN> Attempting to open::../toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read rtf card -
 CHARMM>     name ../toppar/top_all36_cgenff.rtf -
 CHARMM>     append
 VOPEN> Attempting to open::../toppar/top_all36_cgenff.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *  --------------------------------------------------------------------------  *
 TITLE> *          CGENFF: TOPOLOGY FOR THE CHARMM GENERAL FORCE FIELD V. 4.6      

1

In [5]:
## Build protein
protein_psf = os.path.join(proteindir, f'{protein}_fad.psf')
protein_pdb = os.path.join(proteindir, f'{protein}_fad.pdb')
read.psf_card(protein_psf, append = True)
read.pdb(protein_pdb, resid = True)

  
 CHARMM>     read psf card -
 CHARMM>     name ../cofactor/pdb_with_fad/tropb_fad.psf -
 CHARMM>     append
 VOPEN> Attempting to open::../cofactor/pdb_with_fad/tropb_fad.psf::
 MAINIO> Protein structure file being appended from unit  91.
 psf_read_formatted: Reading PSF in the expanded format.
 TITLE>  * EXECUTING CHARMM SCRIPT FROM PYTHON
 TITLE>  *  DATE:    12/10/22     14:18:36      CREATED BY USER: azamh
 TITLE>  *
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        2   Number of residues   =      448
         Number of atoms         =     7040   Number of groups     =     2042
         Number of bonds         =     7130   Number of angles     =    12832
         Number of dihedrals     =    18767   Number of impropers  =     1275
         Number of cross-terms   =      447   Number of autogens   =        0
         Number of HB acceptors  =      652   Number of HB donors  =  

In [6]:
## Generate grids
gridfile = f'grid/{protein}_fftdock.bin'
probefile = os.path.join(toppardir, 'fftdock_c36prot_cgenff_probes.txt')
cdocker = grid.CDOCKER()
settings = {'xCen' : xcen, 'yCen' : ycen, 'zCen' : zcen, 
            'xMax' : maxlen, 'yMax' : maxlen, 'zMax' : maxlen,
            'emax' : 2, 'maxe' : 40, 'mine' : -20, 'flag_gpu' : True, 
            'gridFile' : gridfile, 'dielec' : 3, 'flag_rdie' : True,
            'probeFile' : probefile} 
cdocker.setVar(settings)
cdocker.generate()

 VOPEN> Attempting to open::grid/tropb_fftdock.bin::
Use GPU to generate grids
Use default non-bond set up
  
 CHARMM>     update atom -
 CHARMM>     switch -
 CHARMM>     vswitch -
 CHARMM>     soft -
 CHARMM>     vdwe -
 CHARMM>     elee -
 CHARMM>     rdie -
 CHARMM>     cutnb 999 -
 CHARMM>     ctofnb 999 -
 CHARMM>     ctonnb 999 -
 CHARMM>     emax 2 -
 CHARMM>     mine -20 -
 CHARMM>     maxe 40 -
 CHARMM>     epsilon 3

 **** SOFT CORE AVAILABLE 
       SUGGESTED OPTIONS : RDIE SWIT VSWIT 
 FOR SPC WATER in CDIE USE : EMAX > 1000/EPS OR MINE=-100/EPS,
 FOR SPC WATER in RDIE USE : EMAX > 200/EPS

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
 - VDW core too soft with respect to electrostatics 
 **** INCREASE EMAX to at least       5.00

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
 - Elec attractive soft core starts at too high Rcut
 **** DECREASE MINE to at least     -33.33

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
  - Elec repulsive soft core starts at too high Rcut
 **** INCREASE M

True

In [7]:
## Prepare system
ligand_pdb = os.path.join(liganddir, f'{ligand}.pdb')
psf.delete_atoms(pycharmm.SelectAtoms().all_atoms())
read.sequence_pdb(ligand_pdb)
gen.new_segment(seg_name = "LIGA")
read.pdb(ligand_pdb, resid = True)
minimize.run_sd(nstep=1000, tolenr=1e-3, tolgrd=1e-4)



 Message from MAPIC: Atom numbers are changed.

 Message from MAPIC:        448 residues deleted.

 Message from MAPIC:          2 segments deleted.
 DELTIC:      7130 bonds deleted
 DELTIC:     12832 angles deleted
 DELTIC:     18767 dihedrals deleted
 DELTIC:      1275 improper dihedrals deleted
 DELTIC:       447 crossterm maps deleted
 DELTIC:       790 donors deleted
 DELTIC:       652 acceptors deleted
  
 CHARMM>     read sequence pdb -
 CHARMM>     name ./ligands/2.pdb
 VOPEN> Attempting to open::./ligands/2.pdb::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --     1 RESIDUES
          LIG     
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 NO PATCHING WILL BE DONE ON THE FIRST RESIDUE
 NO PATCHING WILL BE DONE ON THE LAST  RESIDUE
 AUTGEN: Autogenerating specified angles and dihedrals.
 GENPSF> Segment   1 has been generated. Its identifier is LIGA.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms clear

True

In [8]:
#Get initial energy
def get_energy_df(pose_name):
    df = energy.get_energy().set_index('name').drop(columns = 'feature').transpose()
    df.index = [pose_name]
    df.index.name = 'pose'
    df.columns.name = 'term'
    return df
initial_energy_df = get_energy_df('initial')
initial_energy_df


 **** SOFT CORE AVAILABLE 
       SUGGESTED OPTIONS : RDIE SWIT VSWIT 
 FOR SPC WATER in CDIE USE : EMAX > 1000/EPS OR MINE=-100/EPS,
 FOR SPC WATER in RDIE USE : EMAX > 200/EPS

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
 - VDW core too soft with respect to electrostatics 
 **** INCREASE EMAX to at least       5.00

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
 - Elec attractive soft core starts at too high Rcut
 **** DECREASE MINE to at least     -33.33

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
  - Elec repulsive soft core starts at too high Rcut
 **** INCREASE MAXA to at least      66.67

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    RDIElec  SWITch   VATOm    VSWItch 
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  =999.000 CTEXNB =999.000 CTONNB =999.000 CTOFNB =999.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  3.000
 NBXMOD =      5
 VDW SOFT CORE: BEGINS AT: EMAX =      2.00 kcal/mol
 VDW SOFT CORE : LINEAR FORM 
 ELECTROSTATIC A

term       energy  deltae     grms     bonds    angles     ureyb  dihedrals  \
pose                                                                          
initial  9.815651     0.0  1.41683  1.183762  2.072955  0.075093   6.258302   

term     impropers   vdwaals      elec  hbonds  asp  user  
pose                                                       
initial   0.007516  9.622641 -9.404619     0.0  0.0   0.0

In [9]:
#Read grid 
charmm_script(f"open unit 30 read unform name {gridfile}")
charmm_script("fftg read unit 30")

  
 CHARMM>     open unit 30 read unform name grid/tropb_fftdock.bin
 VOPEN> Attempting to open::grid/tropb_fftdock.bin::
 OPNLGU> Unit 30 opened for READONLY access to grid/tropb_fftdock.bin
  
  
 CHARMM>     fftg read unit 30
 Grid potentials read from binary file on unit  30
* EXECUTING CHARMM SCRIPT FROM PYTHON
*  DATE:    12/11/22     15:37:55      CREATED BY USER: azamh
*
 GridSetUp: Grid potentials will be set-up for  26 atom types plus electrostatics
 GridSetUp: and read from unit  30
 GridSetUp: Grid centered at  57.13538  45.17524  46.06176
 GridSetUp: Hydrogen bond grids   0.00000   0.00000   0.00000
 GridSetUp: Grid runs from (X)  51.13538 -  63.13538
 GridSetUp: Grid runs from (Y)  39.17524 -  51.17524
 GridSetUp: Grid runs from (Z)  40.06176 -  52.06176

 GridSetUp: With a grid spacing of   0.50000
 GridSetUp: Force constant at grid edge set to    300.000 kcal/mol/A^2
  


1

In [10]:
#Read list of rotations
quaternion_set = os.path.join(toppardir, 'fftdock_rotation_3.qua')
charmm_script(f"open unit 31 read form name {quaternion_set}")

  
 CHARMM>     open unit 31 read form name ../toppar/fftdock_rotation_3.qua
 VOPEN> Attempting to open::../toppar/fftdock_rotation_3.qua::
 OPNLGU> Unit 31 opened for READONLY access to ../toppar/fftdock_rotation_3.qua
  


1

In [11]:
#Perform FFTDock
nsave = 500
charmm_script(f"fftg lcon ncon 1 icon 1  nrok {nsave} quau 31 sizb 100 select segid LIGA end")
charmm_script("close unit 31")

  
 CHARMM>     fftg lcon ncon 1 icon 1  nrok 500 quau 31 sizb 100 select segid LIGA end
Num of GPU Devices: 1
The device 0 is used. 
  GPU Devices Name: NVIDIA GeForce RTX 2080 Ti
  total global devices memory: 11019 MB
FFTDOCK - num of orientations: 36864
FFTDOCK - batch_size:   100
FFTDOCK - num_batch:   369
 SELRPN>     32 atoms have been selected out of     32
<FFTDock> Initializing ligand grid
Total number of grids =    27
Types of VDW grid used  =     9
BatchId / Total =     1 /   369
BatchId / Total =     2 /   369
BatchId / Total =     3 /   369
BatchId / Total =     4 /   369
BatchId / Total =     5 /   369
BatchId / Total =     6 /   369
BatchId / Total =     7 /   369
BatchId / Total =     8 /   369
BatchId / Total =     9 /   369
BatchId / Total =    10 /   369
BatchId / Total =    11 /   369
BatchId / Total =    12 /   369
BatchId / Total =    13 /   369
BatchId / Total =    14 /   369
BatchId / Total =    15 /   369
BatchId / Total =    16 /   369
BatchId / Total =    17

1

In [12]:
#Load grid for pose score
charmm_script(f'''
open unit 1 read unform name {gridfile}
grid read unit 1 select all end
close unit 1
''')

  
 CHARMM>     
  
 CHARMM>    open unit 1 read unform name grid/tropb_fftdock.bin
 VOPEN> Attempting to open::grid/tropb_fftdock.bin::
 VCLOSE: Closing unit   30 with status "KEEP"
 OPNLGU> Unit  1 opened for READONLY access to grid/tropb_fftdock.bin
  
 CHARMM>    grid read unit 1 select all end
 Grid potentials read from binary file on unit   1
* EXECUTING CHARMM SCRIPT FROM PYTHON
*  DATE:    12/11/22     15:38: 1      CREATED BY USER: azamh
*
 GridSetUp: Grid potentials will be set-up for  26 atom types plus electrostatics
 GridSetUp: and written to unit   1
 GridSetUp: Grid centered at  57.13538  45.17524  46.06176
 GridSetUp: Grid runs from (X)  51.13538 -  63.13538
 GridSetUp: Grid runs from (Y)  39.17524 -  51.17524
 GridSetUp: Grid runs from (Z)  40.06176 -  52.06176

 GridSetUp: With a grid spacing of   0.50000
 GridSetUp: Force constant at grid edge set to    300.000 kcal/mol/A^2
 GridSetUp: Allocating   474552 Real(chm_Real) words for grid potentials.
Hydrogen bond grids 

1

In [13]:
#Go through each rotation
pose_energy_dfs = [initial_energy_df]
for i in range(1, nsave + 1):
    print(i)
    
    #Set rotation
    charmm_script(f'fftg coor icon 1 irot {i}')
    
    #Calculate energy
    pose_energy_df = get_energy_df(i)
    pose_energy_dfs.append(pose_energy_df)
    
    #Save pose
    pose_pdb = os.path.join(dockdir, f'{protein}_{ligand}_{i}.crd')
    write.coor_pdb(pose_pdb)

1
  
 CHARMM>     fftg coor icon 1 irot 1
 Coordinates of the     1th conforamtion and the     1th rotation from FFTDOCK are copied into the main coordinates.
  

 **** SOFT CORE AVAILABLE 
       SUGGESTED OPTIONS : RDIE SWIT VSWIT 
 FOR SPC WATER in CDIE USE : EMAX > 1000/EPS OR MINE=-100/EPS,
 FOR SPC WATER in RDIE USE : EMAX > 200/EPS

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
 - VDW core too soft with respect to electrostatics 
 **** INCREASE EMAX to at least       5.00

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
 - Elec attractive soft core starts at too high Rcut
 **** DECREASE MINE to at least     -33.33

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
  - Elec repulsive soft core starts at too high Rcut
 **** INCREASE MAXA to at least      66.67

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    RDIElec  SWITch   VATOm    VSWItch 
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  =999.000 CTEXNB =999.000 CTONNB =999.000 CTOFNB =999.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WR

In [14]:
#Concat energy dataframes
energy_df = pd.concat(pose_energy_dfs).fillna(0)
energy_df

term        energy     deltae      grms     bonds    angles     ureyb  \
pose                                                                    
initial   9.815651   0.000000  1.416830  1.183762  2.072955  0.075093   
1       -33.187982  43.003633  2.741189  1.183943  2.072970  0.075084   
2       -32.524721  -0.663261  3.959383  1.183835  2.072977  0.075091   
3       -32.452706  -0.072016  2.758132  1.183779  2.072987  0.075093   
4       -32.129328  -0.323378  3.607025  1.183730  2.072972  0.075102   
...            ...        ...       ...       ...       ...       ...   
496     -24.376176  -0.003232  3.347513  1.183724  2.072976  0.075097   
497     -24.369863  -0.006313  3.235397  1.183825  2.072944  0.075095   
498     -24.369790  -0.000072  2.635171  1.183601  2.072981  0.075107   
499     -24.366016  -0.003774  3.086457  1.183616  2.072917  0.075108   
500      23.234647 -47.600663  6.897244  1.183667  2.072924  0.075095   

term     dihedrals  impropers   vdwaals      elec  hbonds  asp  user  \
pose                                                                   
initial   6.258302   0.007516  9.622641 -9.404619     0.0  0.0   0.0   
1         6.258292   0.007521  9.622493 -9.404620     0.0  0.0   0.0   
2         6.258315   0.007513  9.622565 -9.404631     0.0  0.0   0.0   
3         6.258307   0.007514  9.622642 -9.404636     0.0  0.0   0.0   
4         6.258294   0.007522  9.622677 -9.404634     0.0  0.0   0.0   
...            ...        ...       ...       ...     ...  ...   ...   
496       6.258310   0.007513  9.622658 -9.404616     0.0  0.0   0.0   
497       6.258311   0.007513  9.622621 -9.404621     0.0  0.0   0.0   
498       6.258305   0.007517  9.622809 -9.404655     0.0  0.0   0.0   
499       6.258306   0.007516  9.622789 -9.404620     0.0  0.0   0.0   
500       6.258299   0.007512  9.622769 -9.404629     0.0  0.0   0.0   

term         grvdw     grelec  
pose                           
initial   0.000000   0.000000  
1         5.560168 -48.563833  
2         9.686021 -52.026409  
3         2.301406 -44.569798  
4        -5.745652 -36.199338  
...            ...        ...  
496      -0.455320 -33.736518  
497      -6.946387 -27.239163  
498      -7.492673 -26.692782  
499      -1.217109 -32.964539  
500      25.001386 -11.582375  

[501 rows x 15 columns]

In [15]:
#Get final-initial energy
delta_energy_df = energy_df.subtract(energy_df.loc['initial'].values, axis = 1)
delta_energy_df

term        energy     deltae      grms     bonds    angles         ureyb  \
pose                                                                        
initial   0.000000   0.000000  0.000000  0.000000  0.000000  0.000000e+00   
1       -43.003633  43.003633  1.324359  0.000180  0.000015 -8.286088e-06   
2       -42.340372  -0.663261  2.542553  0.000073  0.000022 -1.251815e-06   
3       -42.268356  -0.072016  1.341302  0.000017  0.000031  7.749453e-07   
4       -41.944978  -0.323378  2.190195 -0.000032  0.000016  9.763549e-06   
...            ...        ...       ...       ...       ...           ...   
496     -34.191826  -0.003232  1.930682 -0.000039  0.000021  4.404111e-06   
497     -34.185513  -0.006313  1.818567  0.000062 -0.000011  2.668619e-06   
498     -34.185441  -0.000072  1.218341 -0.000161  0.000026  1.417046e-05   
499     -34.181666  -0.003774  1.669627 -0.000146 -0.000038  1.525494e-05   
500      13.418997 -47.600663  5.480414 -0.000096 -0.000031  2.316705e-06   

term     dihedrals     impropers   vdwaals          elec  hbonds  asp  user  \
pose                                                                          
initial   0.000000  0.000000e+00  0.000000  0.000000e+00     0.0  0.0   0.0   
1        -0.000010  5.252727e-06 -0.000148 -9.486982e-07     0.0  0.0   0.0   
2         0.000012 -2.621886e-06 -0.000075 -1.241283e-05     0.0  0.0   0.0   
3         0.000004 -2.187592e-06  0.000002 -1.723742e-05     0.0  0.0   0.0   
4        -0.000008  5.841576e-06  0.000036 -1.577736e-05     0.0  0.0   0.0   
...            ...           ...       ...           ...     ...  ...   ...   
496       0.000008 -2.387272e-06  0.000018  2.973864e-06     0.0  0.0   0.0   
497       0.000008 -3.213555e-06 -0.000019 -2.391364e-06     0.0  0.0   0.0   
498       0.000003  7.801311e-07  0.000168 -3.623520e-05     0.0  0.0   0.0   
499       0.000003  4.245000e-08  0.000149 -9.718159e-07     0.0  0.0   0.0   
500      -0.000003 -3.526013e-06  0.000128 -1.051428e-05     0.0  0.0   0.0   

term         grvdw     grelec  
pose                           
initial   0.000000   0.000000  
1         5.560168 -48.563833  
2         9.686021 -52.026409  
3         2.301406 -44.569798  
4        -5.745652 -36.199338  
...            ...        ...  
496      -0.455320 -33.736518  
497      -6.946387 -27.239163  
498      -7.492673 -26.692782  
499      -1.217109 -32.964539  
500      25.001386 -11.582375  

[501 rows x 15 columns]

In [16]:
#Save energies
scorefile = f'scores/{protein}_{ligand}_fftdock.csv'
delta_energy_df.to_csv(scorefile)